# <span style="color:green;">Import </span>

In [125]:
#Scrap 

import requests #Requests allows to send HTTP/1.1 requests

#Utilitaires 
import json
import pandas as pd
import numpy as np
from pandas import json_normalize

from concurrent.futures import ThreadPoolExecutor

#Neo4j

from neo4j import GraphDatabase
from py2neo import Graph
from neomodel import StructuredNode, StringProperty, RelationshipTo, RelationshipFrom, config

import dotenv
import os

# <span style="color:brown;">Scrap Data</span>

In [4]:
occ_url = "https://ec.europa.eu/esco/api/search?language=en&isInScheme=http://data.europa.eu/esco/concept-scheme/occupations&limit=3561"

In [5]:
response = requests.get(occ_url) #Store the request into a new variable

In [6]:
Occupation_source = response.json()

In [7]:
Occupation = Occupation_source['_embedded']

In [8]:
df_Occupation = pd.DataFrame(Occupation["results"]) #Convert to pandas

In [9]:
df_Occupation

,className,classId,uri,title,preferredLabel,code,isInScheme,broaderOccupation,_links,isTopConceptInScheme,broaderIscoGroup,broaderConcept
0,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/52df9d56...,3D animator,{'de': '3D-Animationstechniker/3D-Animationste...,2166.3.1,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/occupation/48c8ecc...,{'self': {'href': 'https://ec.europa.eu/esco/a...,NaN,NaN,NaN
1,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/bab5fa79...,3D modeller,"{'de': '3D-Artist/3D-Artistin', 'no': '3D-mode...",2166.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2166],NaN
2,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/4cf7be91...,3D printing technician,{'de': '3D-Druck-Techniker/3D-Druck-Technikeri...,3118.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C3118],NaN
3,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/3c0af499...,abrasive blasting operator,"{'de': 'Sandstrahler/Sandstrahlerin', 'no': 'o...",8122.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C8122],NaN
4,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/c706886d...,absorbent pad machine operator,{'de': 'Bediener von Maschinen zur Erzeugung v...,8143.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C8143],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3556,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/7c5e4ca1...,zoo educator,"{'de': 'Zoopädagoge/Zoopädagogin', 'no': 'unde...",5113.1.3,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/occupation/b476f45...,{'self': {'href': 'https://ec.europa.eu/esco/a...,NaN,NaN,NaN
3557,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/e7417f4f...,zoo registrar,"{'de': 'Wissenschaftlicher Zoomitarbeiter, ver...",2621.9,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2621],NaN
3558,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/62204bd8...,zoo section leader,{'de': 'Cheftierpfleger Zoo/Cheftierpflegerin ...,5164.10,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C5164],NaN
3559,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/f7066fc0...,zookeeper,"{'de': 'Tierpfleger Zoo/Tierpflegerin Zoo', 'n...",5164.1.3,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/occupation/0b14d5d...,{'self': {'href': 'https://ec.europa.eu/esco/a...,NaN,NaN,NaN


# <span style="color:brown;">Define List of necessary jobs</span>

In [68]:
DataJobsIndices = df_Occupation['title'].str.contains('clean', na=False)
df_DataJob = df_Occupation[DataJobsIndices]

This code snippet is a filter to include only rows where the 'title' column contains certain keywords related to clean.

In [69]:
print(len(df_DataJob))

21


In [70]:
df_DataJob['title']

120                                amusement park cleaner
354     Bleaching, dyeing and fabric cleaning machine ...
411                                      building cleaner
415                             building exterior cleaner
422                           Building structure cleaners
458                                   cacao beans cleaner
932                                      domestic cleaner
933                         Domestic cleaners and helpers
938       Domestic, hotel and office cleaners and helpers
951                            drapery and carpet cleaner
1334                                    furniture cleaner
1841                     laundry and dry cleaning manager
2332                               Other cleaning workers
2357    Painters, building structure cleaners and rela...
2904                                     sewerage cleaner
3291                                        train cleaner
3361                                      vehicle cleaner
3362          

In [71]:
#Occupation list from Julien 
# Nec_Occ = ['data analyst',
# 'data scientist',
# 'computer scientist',
# 'chief data officer',
# 'ICT research consultant',
# 'data centre operator',
# 'ICT business analyst',
# 'data quality specialist',
# 'data warehouse designer',
# 'business analyst','data protection officer','database administrator','database designer','database integrator',
# 'database developer','ICT security manager','knowledge engineer','test engineer','mathematician research engineer',
# 'bioinformatics scientist']

In [72]:
df_DataJob = df_DataJob[df_DataJob['className'].isin(['Occupation'])]
# df_DataJob = df_DataJob.reset_index()

In [73]:
df_DataJob

,className,classId,uri,title,preferredLabel,code,isInScheme,broaderOccupation,_links,isTopConceptInScheme,broaderIscoGroup,broaderConcept
120,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/38eb308f...,amusement park cleaner,{'de': 'Reinigungs- und Wartungskraft Freizeit...,5153.1.1,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/occupation/1822bfc...,{'self': {'href': 'https://ec.europa.eu/esco/a...,NaN,NaN,NaN
411,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/303a1e34...,building cleaner,{'de': 'Gebäudereiniger Innenreinigung/Gebäude...,9112.2,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C9112],NaN
415,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/a632fb74...,building exterior cleaner,"{'de': 'Fassadenreiniger/Fassadenreinigerin', ...",7133.2,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C7133],NaN
458,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/5520bfd0...,cacao beans cleaner,"{'de': 'Kakaobohnenreinigungskraft', 'no': 'ma...",8160.10,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C8160],NaN
932,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/cbad55a4...,domestic cleaner,"{'de': 'Reinigungskraft in Privathaushalt', 'n...",9111.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C9111],NaN
951,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/f8438d21...,drapery and carpet cleaner,{'de': 'Teppich- und Gardinenreiniger/Teppich-...,9129.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C9129],NaN
1334,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/23be4cb6...,furniture cleaner,"{'de': 'Möbelreiniger/Möbelreinigerin', 'no': ...",9112.3,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C9112],NaN
1841,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/44cd9d91...,laundry and dry cleaning manager,{'de': 'Leiter einer Wäscherei oder Reinigung/...,1439.4,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C1439],NaN
2904,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/1f31c047...,sewerage cleaner,"{'de': 'Kanalreiniger/Kanalreinigerin', 'no': ...",9129.2,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C9129],NaN
3291,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/cb932c74...,train cleaner,{'de': 'Schienenfahrzeugreiniger/Schienenfahrz...,9112.6,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C9112],NaN


In [74]:
df_DataJob['title']

120               amusement park cleaner
411                     building cleaner
415            building exterior cleaner
458                  cacao beans cleaner
932                     domestic cleaner
951           drapery and carpet cleaner
1334                   furniture cleaner
1841    laundry and dry cleaning manager
2904                    sewerage cleaner
3291                       train cleaner
3361                     vehicle cleaner
3518                      window cleaner
Name: title, dtype: object

# Data Exploration

In [75]:
df_DataJob['_links'].iloc[1]

{'self': {'href': 'https://ec.europa.eu/esco/api/resource/occupation?uri=http://data.europa.eu/esco/occupation/303a1e34-cb16-4054-b323-81e5eec17397&language=en',
  'uri': 'http://data.europa.eu/esco/occupation/303a1e34-cb16-4054-b323-81e5eec17397',
  'title': 'building cleaner'}}

In the given JSON object, 'href' and 'uri' are used to represent the same resource, which is an occupation with the title of "bioinformatics scientist". The 'uri' provides a unique identifier for the occupation, while the 'href' provides the URL where the occupation's information can be accessed.

In [76]:
# python
df_DataJob = (
    df_DataJob.loc[DataJobsIndices]
    .assign(_links_href=lambda d: d['_links'].apply(lambda x: x['self']['href']))
)

In [77]:
df_DataJob['_links_href']

120     https://ec.europa.eu/esco/api/resource/occupat...
411     https://ec.europa.eu/esco/api/resource/occupat...
415     https://ec.europa.eu/esco/api/resource/occupat...
458     https://ec.europa.eu/esco/api/resource/occupat...
932     https://ec.europa.eu/esco/api/resource/occupat...
951     https://ec.europa.eu/esco/api/resource/occupat...
1334    https://ec.europa.eu/esco/api/resource/occupat...
1841    https://ec.europa.eu/esco/api/resource/occupat...
2904    https://ec.europa.eu/esco/api/resource/occupat...
3291    https://ec.europa.eu/esco/api/resource/occupat...
3361    https://ec.europa.eu/esco/api/resource/occupat...
3518    https://ec.europa.eu/esco/api/resource/occupat...
Name: _links_href, dtype: object

In [78]:
df_DataJob

,className,classId,uri,title,preferredLabel,code,isInScheme,broaderOccupation,_links,isTopConceptInScheme,broaderIscoGroup,broaderConcept,_links_href
120,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/38eb308f...,amusement park cleaner,{'de': 'Reinigungs- und Wartungskraft Freizeit...,5153.1.1,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/occupation/1822bfc...,{'self': {'href': 'https://ec.europa.eu/esco/a...,NaN,NaN,NaN,https://ec.europa.eu/esco/api/resource/occupat...
411,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/303a1e34...,building cleaner,{'de': 'Gebäudereiniger Innenreinigung/Gebäude...,9112.2,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C9112],NaN,https://ec.europa.eu/esco/api/resource/occupat...
415,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/a632fb74...,building exterior cleaner,"{'de': 'Fassadenreiniger/Fassadenreinigerin', ...",7133.2,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C7133],NaN,https://ec.europa.eu/esco/api/resource/occupat...
458,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/5520bfd0...,cacao beans cleaner,"{'de': 'Kakaobohnenreinigungskraft', 'no': 'ma...",8160.10,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C8160],NaN,https://ec.europa.eu/esco/api/resource/occupat...
932,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/cbad55a4...,domestic cleaner,"{'de': 'Reinigungskraft in Privathaushalt', 'n...",9111.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C9111],NaN,https://ec.europa.eu/esco/api/resource/occupat...
951,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/f8438d21...,drapery and carpet cleaner,{'de': 'Teppich- und Gardinenreiniger/Teppich-...,9129.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C9129],NaN,https://ec.europa.eu/esco/api/resource/occupat...
1334,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/23be4cb6...,furniture cleaner,"{'de': 'Möbelreiniger/Möbelreinigerin', 'no': ...",9112.3,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C9112],NaN,https://ec.europa.eu/esco/api/resource/occupat...
1841,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/44cd9d91...,laundry and dry cleaning manager,{'de': 'Leiter einer Wäscherei oder Reinigung/...,1439.4,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C1439],NaN,https://ec.europa.eu/esco/api/resource/occupat...
2904,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/1f31c047...,sewerage cleaner,"{'de': 'Kanalreiniger/Kanalreinigerin', 'no': ...",9129.2,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C9129],

In [79]:
response = requests.get(df_DataJob['_links_href'].iloc[0])
response = response.json()

In [80]:
response.keys()

dict_keys(['className', 'classId', 'uri', 'title', 'referenceLanguage', 'preferredLabel', 'alternativeLabel', 'preferredTerm', 'alternativeTerms', 'description', 'status', 'code', 'codes', '_links', '_embedded'])

In [81]:
response['_links']

{'self': {'href': 'https://ec.europa.eu/esco/api/resource/occupation?uri=http://data.europa.eu/esco/occupation/38eb308f-0075-451e-9c1a-c557fd173022&language=en',
  'uri': 'http://data.europa.eu/esco/occupation/38eb308f-0075-451e-9c1a-c557fd173022',
  'title': 'amusement park cleaner'},
 'isInScheme': [{'href': 'https://ec.europa.eu/esco/api/resource/taxonomy?uri=http://data.europa.eu/esco/concept-scheme/occupations&language=en',
   'uri': 'http://data.europa.eu/esco/concept-scheme/occupations',
   'title': 'ESCO Occupations'},
  {'href': 'https://ec.europa.eu/esco/api/resource/taxonomy?uri=http://data.europa.eu/esco/concept-scheme/member-occupations&language=en',
   'uri': 'http://data.europa.eu/esco/concept-scheme/member-occupations',
   'title': 'ESCO member occupations'}],
 'regulatedProfessionNote': {'href': 'https://ec.europa.eu/esco/api/resource/concept?uri=http://data.europa.eu/esco/regulated-professions/unregulated&language=en',
  'uri': 'http://data.europa.eu/esco/regulated-pr

In [82]:
links = response['_links']

In [83]:
links.keys()

dict_keys(['self', 'isInScheme', 'regulatedProfessionNote', 'broaderOccupation', 'hasEssentialSkill', 'hasOptionalSkill'])

In [84]:
ess_skills = links['hasEssentialSkill']
opt_skills = links['hasOptionalSkill']

In [85]:
ess_skills

[{'href': 'https://ec.europa.eu/esco/api/resource/skill?uri=http://data.europa.eu/esco/skill/d05ae693-dc7d-4073-8806-2782f117c24a&language=en',
  'uri': 'http://data.europa.eu/esco/skill/d05ae693-dc7d-4073-8806-2782f117c24a',
  'title': 'clean glass surfaces',
  'skillType': 'http://data.europa.eu/esco/skill-type/skill'},
 {'href': 'https://ec.europa.eu/esco/api/resource/skill?uri=http://data.europa.eu/esco/skill/b9a9459c-befa-4b30-95b8-25b5dc83d8f1&language=en',
  'uri': 'http://data.europa.eu/esco/skill/b9a9459c-befa-4b30-95b8-25b5dc83d8f1',
  'title': 'clean amusement park facilities',
  'skillType': 'http://data.europa.eu/esco/skill-type/skill'},
 {'href': 'https://ec.europa.eu/esco/api/resource/skill?uri=http://data.europa.eu/esco/skill/c585a661-1720-4138-8cc7-0029452a8534&language=en',
  'uri': 'http://data.europa.eu/esco/skill/c585a661-1720-4138-8cc7-0029452a8534',
  'title': 'amusement park emergency procedures',
  'skillType': 'http://data.europa.eu/esco/skill-type/knowledge'}

In [86]:
ess_skills[0]['title'] , ess_skills[0]['skillType']

('clean glass surfaces', 'http://data.europa.eu/esco/skill-type/skill')

# Skills DF

In [87]:
# Create a list to store the data
data = []

# Loop through the rows of the original DataFrame
for i, row in df_DataJob.iterrows():
    # Get the href value from the _links column
    href = row['_links']['self']['href']
    # Send a request to the API endpoint and get the JSON response
    response = requests.get(href).json()
    # Get the essential and optional skills from the JSON response
    ess_skills = response['_links']['hasEssentialSkill']
    opt_skills = response['_links']['hasOptionalSkill']
    # Get the title from the original DataFrame
    title = row['title']
   # Loop through the essential skills and append them to the "data" list
    for skill in ess_skills:
        skill_type = skill['skillType']
        data.append({'Title': title, 'Href': href, 'Relation': 'essential', 'Skill_title': skill['title'], 'Skill_type': skill_type})
# Loop through the optional skills and append them to the "data" list
    for skill in opt_skills:
        skill_type = skill['skillType']
        data.append({'Title': title, 'Href': href, 'Relation': 'optional', 'Skill_title': skill['title'], 'Skill_type': skill_type})

# Create a new DataFrame from the data list
df_skills = pd.DataFrame(data)

## Clean df

In [88]:
# Get the last part of the string after the last "/"
df_skills['Skill_type'] = df_skills['Skill_type'].apply(lambda x: x.split("/")[-1])
df_skills

,Title,Href,Relation,Skill_title,Skill_type
0,amusement park cleaner,https://ec.europa.eu/esco/api/resource/occupat...,essential,maintain amusement park attractions,skill
1,amusement park cleaner,https://ec.europa.eu/esco/api/resource/occupat...,essential,clean glass surfaces,skill
2,amusement park cleaner,https://ec.europa.eu/esco/api/resource/occupat...,essential,maintain amusement park equipment,skill
3,amusement park cleaner,https://ec.europa.eu/esco/api/resource/occupat...,essential,clean amusement park facilities,skill
4,amusement park cleaner,https://ec.europa.eu/esco/api/resource/occupat...,essential,perform minor repairs to equipment,skill
...,...,...,...,...,...
376,window cleaner,https://ec.europa.eu/esco/api/resource/occupat...,optional,supervise the work of cleaning staff,skill
377,window cleaner,https://ec.europa.eu/esco/api/resource/occupat...,optional,manage a small-to-medium business,skill
378,window cleaner,https://ec.europa.eu/esco/api/resource/occupat...,optional,perform cleaning activities in an environmenta...,skill
379,window cleaner,https://ec.europa.eu/esco/api/resource/occupat...,optional,follow safety procedures when working at heights,skill


# Occupation DF
for main node

In [89]:
response.keys()

dict_keys(['className', 'classId', 'uri', 'title', 'referenceLanguage', 'preferredLabel', 'alternativeLabel', 'preferredTerm', 'alternativeTerms', 'description', 'status', 'code', 'codes', '_links', '_embedded'])

In [90]:
response['title'], response['preferredLabel']['fr'], response['alternativeLabel']['en'], response['description']['en']


('window cleaner',
 'laveur de vitres/laveuse de vitres',
 ['building maintenance and cleaning worker', 'window washer', 'janitress'],
 {'literal': 'Window cleaners use cleaning tools such as sponges and detergents to clean windows, mirrors and other glass surfaces of buildings, both on the interior and exterior. They use specific ladders to clean taller buildings, using safety belts for support.',
  'mimetype': 'plain/text'})

In [91]:
response['alternativeLabel']['en']

['building maintenance and cleaning worker', 'window washer', 'janitress']

In [109]:
# Create an empty list to store the data
data = []

# Iterate over the _links_href column and extract the required data
for href in df_DataJob['_links_href']:
    # Send a request to the API endpoint and get the JSON response
    response = requests.get(href).json()
    # Extract the required data from the response
    title = response['title']
    pref_label = response['preferredLabel']['en']
    # Check if 'alternativeLabel' exists and is not empty
    if 'alternativeLabel' in response and "en" in response.get("alternativeLabel", {}):
        # Join the alternative labels into a single string
        alt_label = ", ".join([label['value'] if isinstance(label, dict) else label for label in response['alternativeLabel']['en']])
    else:
        alt_label = ""
    desc = response['description']['en']
    # Append the data to the "data" list
    data.append({'Title': title, 'Preferredlabel': pref_label, 'AltLabel': alt_label, 'Description': desc})

# Create a new DataFrame from the data list
df_new = pd.DataFrame(data, columns=['Title', 'Preferredlabel', 'AltLabel', 'Description'])



In [110]:
df_new['Description'][0]

{'literal': 'Amusement park cleaners work to keep the amusement park clean and take on small repairs. Amusement park cleaners\xa0usually work at night, when the park is closed, but urgent maintenance and cleaning is done during the day.',
 'mimetype': 'plain/text'}

In [111]:
# clean the 'Description' column
for index, row in df_new.iterrows():
    description = row['Description']['literal']
    cleaned_description = description.replace('\'', '').replace('{', '').replace('}', '').replace('literal:', '').replace('mimetype:plain/text,', '')
    df_new.at[index, 'Description'] = cleaned_description

df_new['Description'][0]

'Amusement park cleaners work to keep the amusement park clean and take on small repairs. Amusement park cleaners\xa0usually work at night, when the park is closed, but urgent maintenance and cleaning is done during the day.'

In [112]:
df_new['Description'][0]

'Amusement park cleaners work to keep the amusement park clean and take on small repairs. Amusement park cleaners\xa0usually work at night, when the park is closed, but urgent maintenance and cleaning is done during the day.'

In [114]:
df_occupation = df_new

,Title,Preferredlabel,AltLabel,Description
0,amusement park cleaner,amusement park cleaner,"janitorial worker, cleaners, amusement park ja...",Amusement park cleaners work to keep the amuse...
1,building cleaner,building cleaner,"super, maintenance worker, superintendent, off...",Building cleaners maintain the cleanliness and...
2,building exterior cleaner,building exterior cleaner,"pressure washer operator, façade cleaner, buil...",Building exterior cleaners remove dirt and lit...
3,cacao beans cleaner,cacao beans cleaner,"cocoa bean shelling machine operator, bean cle...",Cacao beans cleaners operate machines for the ...
4,domestic cleaner,domestic cleaner,"cleaning maid, cleaning lady, personal maid, c...",Domestic cleaners perform all necessary cleani...
5,drapery and carpet cleaner,drapery and carpet cleaner,"drapery cleaner, carpet and drapery cleaner, c...",Drapery and carpet cleaners clean draperies an...
6,furniture cleaner,furniture cleaner,"upholstery cleaner, re-upholsterer",Furniture cleaners maintain furniture items by...
7,laundry and dry cleaning manager,laundry and dry cleaning manager,"dry cleaning and laundry supervisor, laundry a...",Laundry and dry cleaning managers oversee the ...
8,sewerage cleaner,sewerage cleaner,,Sewerage cleaners maintain and clean sewerage ...
9,train cleaner,train cleaner,"train cleaning operative, train and station cl...",Train cleaners keep the interiors of trains ti...


# Final DF

In [115]:
df_skills

,Title,Href,Relation,Skill_title,Skill_type
0,amusement park cleaner,https://ec.europa.eu/esco/api/resource/occupat...,essential,maintain amusement park attractions,skill
1,amusement park cleaner,https://ec.europa.eu/esco/api/resource/occupat...,essential,clean glass surfaces,skill
2,amusement park cleaner,https://ec.europa.eu/esco/api/resource/occupat...,essential,maintain amusement park equipment,skill
3,amusement park cleaner,https://ec.europa.eu/esco/api/resource/occupat...,essential,clean amusement park facilities,skill
4,amusement park cleaner,https://ec.europa.eu/esco/api/resource/occupat...,essential,perform minor repairs to equipment,skill
...,...,...,...,...,...
376,window cleaner,https://ec.europa.eu/esco/api/resource/occupat...,optional,supervise the work of cleaning staff,skill
377,window cleaner,https://ec.europa.eu/esco/api/resource/occupat...,optional,manage a small-to-medium business,skill
378,window cleaner,https://ec.europa.eu/esco/api/resource/occupat...,optional,perform cleaning activities in an environmenta...,skill
379,window cleaner,https://ec.europa.eu/esco/api/resource/occupat...,optional,follow safety procedures when working at heights,skill


In [116]:
df_occupation

,Title,Preferredlabel,AltLabel,Description
0,amusement park cleaner,amusement park cleaner,"janitorial worker, cleaners, amusement park ja...",Amusement park cleaners work to keep the amuse...
1,building cleaner,building cleaner,"super, maintenance worker, superintendent, off...",Building cleaners maintain the cleanliness and...
2,building exterior cleaner,building exterior cleaner,"pressure washer operator, façade cleaner, buil...",Building exterior cleaners remove dirt and lit...
3,cacao beans cleaner,cacao beans cleaner,"cocoa bean shelling machine operator, bean cle...",Cacao beans cleaners operate machines for the ...
4,domestic cleaner,domestic cleaner,"cleaning maid, cleaning lady, personal maid, c...",Domestic cleaners perform all necessary cleani...
5,drapery and carpet cleaner,drapery and carpet cleaner,"drapery cleaner, carpet and drapery cleaner, c...",Drapery and carpet cleaners clean draperies an...
6,furniture cleaner,furniture cleaner,"upholstery cleaner, re-upholsterer",Furniture cleaners maintain furniture items by...
7,laundry and dry cleaning manager,laundry and dry cleaning manager,"dry cleaning and laundry supervisor, laundry a...",Laundry and dry cleaning managers oversee the ...
8,sewerage cleaner,sewerage cleaner,,Sewerage cleaners maintain and clean sewerage ...
9,train cleaner,train cleaner,"train cleaning operative, train and station cl...",Train cleaners keep the interiors of trains ti...


# <span style="color:blue;">Neo4j</span>

In [128]:
class Neo4jConnection: #Init class
    '''   
    Python class called Neo4jConnection that can be used to connect to a Neo4j database 
    using the official Python driver GraphDatabase.driver.
    The __init__ method initializes the class with the database URI, username, and password. 
    The close method closes the database driver if it is open. 
    The query method executes a Cypher query with optional parameters and returns the query response as a list.
 
    '''
    
    def __init__(self):
        self.__driver = None
        load_status = dotenv.load_dotenv("./configs/Neo4j-f2d2bee2-Created-2025-08-07.txt")
        if load_status is False:
            raise RuntimeError('Environment variables not loaded.')

        URI = os.getenv("NEO4J_URI")
        AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

        with GraphDatabase.driver(URI, auth=AUTH) as driver:
            driver.verify_connectivity()
            self.__driver = driver
            print("Connection established.")
        if self.__driver is None:
            print("Failed to connect to the database.")

        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


In [129]:
#Connect to DB
conn = Neo4jConnection()

Connection established.


# <span style="color:blue;">Clean db</span>

In [130]:
#Don't run (only to reset db)
clean = conn.query("MATCH (n) DETACH DELETE n ")
clean

/var/folders/_l/y178vbp95glb_qq6ygw8dm8r0000gn/T/ipykernel_9206/3802900042.py:37: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.__driver.session(database=db) if db is not None else self.__driver.session()


[]

# <span style="color:blue;">Create Label</span>

In [131]:
conn.query('CREATE CONSTRAINT title IF NOT EXISTS FOR (t:Title)     REQUIRE t.title IS UNIQUE')
conn.query('CREATE CONSTRAINT skills IF NOT EXISTS FOR (s:Skills)     REQUIRE s.skills IS UNIQUE')
conn.query('CREATE CONSTRAINT knowledge IF NOT EXISTS FOR (k:Knowledge)     REQUIRE k.knowledge IS UNIQUE')

/var/folders/_l/y178vbp95glb_qq6ygw8dm8r0000gn/T/ipykernel_9206/3802900042.py:37: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.__driver.session(database=db) if db is not None else self.__driver.session()


[]

## <span style="color:blue;">Title</span>

In [132]:
def add_title(title): 
    """
    This is a Python function called add_title that takes in a pandas DataFrame 
    and executes a Cypher query on a Neo4j database connected by a conn object 
    
    The Cypher query in this function is designed to create a Title node in the database.
    With properties title, description, and alternativeLabel. 
    
    """
    
    query = '''
        UNWIND $rows AS row
        MERGE (t:Title {title: row.Title})
        ON CREATE SET t.description = row.Description, t.alternativeLabel = row.AltLabel,
        t.preferredLabelFR = row.PreferredlabelinFR
        RETURN count(*) as total
        '''

    #The ON CREATE SET clause sets the description and alternativeLabel properties for the newly created Title node.
    return conn.query(query, parameters={'rows': title.to_dict('records')})

In [133]:
add_title(df_occupation)

/var/folders/_l/y178vbp95glb_qq6ygw8dm8r0000gn/T/ipykernel_9206/3802900042.py:37: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.__driver.session(database=db) if db is not None else self.__driver.session()


[<Record total=12>]

## <span style="color:blue;">Skills/Knowledge</span>

In [134]:
def create_skill_nodes(conn, df_skills):
    """This is a Python function called create_skill_nodes that takes in a pandas DataFrame 
    and executes  Cypher queries on a Neo4j database connected by a conn object 
    
    The Cypher queries in this function are designed to create a Skills and Knowledge nodes in the database.
    With properties href and "essential" or "optional" relation to the nodes t:Title. """
    
    for _, row in df_skills.iterrows():
        title_name = row['Title']
        skill_title = row['Skill_title']
        skill_type = row['Skill_type']
        relation = row['Relation']
        href = row['Href']

        # Determine the node label based on the skill type
        if skill_type == 'skill':
            node_label = 'Skills'
        elif skill_type == 'knowledge':
            node_label = 'Knowledge'
        else:
            raise ValueError(f"Invalid skill type: {skill_type}")

        # First, check if the skill already exists in the database
        existing_skills = conn.query(f"MATCH (s:{node_label} {{skills: $skill_title}}) RETURN s", {'skill_title': skill_title})
        if existing_skills:
            skill_node = existing_skills[0]['s']
        else:
            # If the skill doesn't exist, create a new node
            skill_node = conn.query(f"CREATE (s:{node_label} {{skills: $skill_title, Href: $href}}) RETURN s", {'skill_title': skill_title, 'href': href})[0]['s']

        # Get the Title node
        title_node = conn.query("MATCH (t:Title {title: $title_name}) RETURN t", {'title_name': title_name})[0]['t']

        # Create the relationship between the Title and the skill node
        conn.query("MATCH (t:Title {title: $title_name}), (s) WHERE ID(s) = $skill_id CREATE (t)-[r:" + relation + "]->(s)", {'title_name': title_name, 'skill_id': skill_node.id})


In [135]:
#Take time
create_skill_nodes(conn, df_skills)

/var/folders/_l/y178vbp95glb_qq6ygw8dm8r0000gn/T/ipykernel_9206/3802900042.py:37: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  session = self.__driver.session(database=db) if db is not None else self.__driver.session()
/var/folders/_l/y178vbp95glb_qq6ygw8dm8r0000gn/T/ipykernel_9206/4174884690.py:35: DeprecationWarning: `id` is deprecated, use `element_id` instead
  conn.query("MATCH (t:Title {title: $title_name}), (s) WHERE ID(s) = $skill_id CREATE (t)-[r:" + relation + "]->(s)", {'title_name': title_name, 'skill_id': skill_node.id})
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function. ('id' has been replaced by 'elementId or consider using an application-generated id')} {position: 